In [1]:
import torch
from transformers import AlbertModel, AlbertTokenizer
import re
import os
import requests
from tqdm.auto import tqdm

In [2]:

tokenizer = AlbertTokenizer.from_pretrained("Rostlab/prot_albert", do_lower_case=False)

In [3]:
model = AlbertModel.from_pretrained("Rostlab/prot_albert")

Some weights of the model checkpoint at Rostlab/prot_albert were not used when initializing AlbertModel: ['sop_classifier.classifier.bias', 'predictions.bias', 'predictions.dense.bias', 'predictions.LayerNorm.weight', 'predictions.LayerNorm.bias', 'predictions.decoder.weight', 'sop_classifier.classifier.weight', 'predictions.decoder.bias', 'predictions.dense.weight']
- This IS expected if you are initializing AlbertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [4]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
model = model.eval()

In [5]:
sequences_Example = ["A E T C Z A O","S K T Z P"]

In [6]:
sequences_Example = [re.sub(r"[UZOB]", "X", sequence) for sequence in sequences_Example]

In [7]:
ids = tokenizer.batch_encode_plus(sequences_Example, add_special_tokens=True, pad_to_max_length=True)

/cephyr/users/anwer/Alvis/.local/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:2323: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [8]:
input_ids = torch.tensor(ids['input_ids']).to(device)
attention_mask = torch.tensor(ids['attention_mask']).to(device)

In [9]:

with torch.no_grad():
    embedding = model(input_ids=input_ids,attention_mask=attention_mask)[0]

In [25]:

features = [] 
for seq_num in range(len(embedding)):
    seq_len = (attention_mask[seq_num] == 1).sum()
    seq_emd = embedding[seq_num][1:seq_len-1]
    features.append(seq_emd)

In [26]:
features

[array([[ 0.04881065,  0.0173279 , -0.06056878, ...,  0.03967572,
          0.06766801, -0.02328246],
        [ 0.04336544,  0.02077006, -0.0606331 , ...,  0.01556444,
          0.0737777 , -0.03080053],
        [ 0.04282778,  0.0218733 , -0.06006568, ...,  0.01582877,
          0.07274354, -0.03015471],
        ...,
        [ 0.04327178,  0.0184369 , -0.06224973, ...,  0.01926246,
          0.0711671 , -0.02991483],
        [ 0.03798865,  0.04147185, -0.01904989, ...,  0.02265862,
          0.0638179 , -0.04559465],
        [ 0.04313156,  0.02955717, -0.06327148, ...,  0.00930989,
          0.0831347 , -0.02575161]], dtype=float32),
 array([[-0.03264696,  0.01471225, -0.02923778, ..., -0.11908071,
         -0.11510337,  0.07991273],
        [-0.04346962,  0.02021889, -0.02541854, ..., -0.12042315,
         -0.10899493,  0.09714412],
        [-0.04365508,  0.0226448 , -0.02413503, ..., -0.12346406,
         -0.10826724,  0.10378322],
        [-0.04474265,  0.02288034, -0.02274427, ...,

In [29]:
# embedding = embedding.cpu().numpy()
features_1 = [] 
for seq_num in range(len(embedding)):
    seq_len = (attention_mask[seq_num] == 1).sum()
    seq_emd = embedding[seq_num][1:seq_len-1]
    features_1.append(seq_emd)

In [32]:
features_1

[array([[ 0.04881065,  0.0173279 , -0.06056878, ...,  0.03967572,
          0.06766801, -0.02328246],
        [ 0.04336544,  0.02077006, -0.0606331 , ...,  0.01556444,
          0.0737777 , -0.03080053],
        [ 0.04282778,  0.0218733 , -0.06006568, ...,  0.01582877,
          0.07274354, -0.03015471],
        ...,
        [ 0.04327178,  0.0184369 , -0.06224973, ...,  0.01926246,
          0.0711671 , -0.02991483],
        [ 0.03798865,  0.04147185, -0.01904989, ...,  0.02265862,
          0.0638179 , -0.04559465],
        [ 0.04313156,  0.02955717, -0.06327148, ...,  0.00930989,
          0.0831347 , -0.02575161]], dtype=float32),
 array([[-0.03264696,  0.01471225, -0.02923778, ..., -0.11908071,
         -0.11510337,  0.07991273],
        [-0.04346962,  0.02021889, -0.02541854, ..., -0.12042315,
         -0.10899493,  0.09714412],
        [-0.04365508,  0.0226448 , -0.02413503, ..., -0.12346406,
         -0.10826724,  0.10378322],
        [-0.04474265,  0.02288034, -0.02274427, ...,

In [35]:
features_1.shape

AttributeError: 'list' object has no attribute 'shape'